## [Day 19](https://adventofcode.com/2020/day19)

So this one has a series of instructions which we're to check if match strings in the 'messages' given. They look an awful lot like regular expressions where a pipe means 'or'. It also seems to have a tree like structure in that there are various nodes with possible branches from it.

Our ultimate goal is to check each message against rule zero

In [10]:
import pandas as pd
import numpy as np
import itertools, re, copy
imp = open('../inputs/d19.txt').read().splitlines()
instructions = imp[:imp.index('')]
messages = imp[imp.index('')+1:]
instructions[:5]

['50: 5 18 | 61 39',
 '79: 118 18 | 125 39',
 '102: 38 39 | 19 18',
 '114: 39 38 | 18 100',
 '119: 39 103']

In [11]:
messages[:5]

['abbbbabbbaabababaaabbaabbbbbbbba',
 'bbbabbbbabbabbbbabbbabab',
 'aabbababbbbababbaaabbabb',
 'aababbaaaabbbaaaaabaabbb',
 'aaaabaaababbbbbbbbbbabaa']

In [12]:
# Okay so I think I want to turn all these instructions into lists of tuples separated by the pipes if they exist

keys = [x.split(':')[0] for x in instructions]
values = [x.split(':')[1][1:] for x in instructions]

def split_to_tuple(x):
    ret = []
    pieces = x.split('|')
    for piece in pieces:
        piece = piece.strip().split(' ')
        piece = [x.replace('"', '') for x in piece]
        ret.append(piece)
    return ret
values = [split_to_tuple(x) for x in values]
instructions = dict(zip(keys, values))
# save a copy
instructions_old = copy.deepcopy(instructions)
instructions['50']

[['5', '18'], ['61', '39']]

In [13]:
# Really struggling with whether this will be easier to start with the zero-th instruction
# and go downwards or start with the ones that have and then move upwards....
# I think i'll try the latter even though I'm having some doubts about whether there 
# are some situations where I'll end up with multiple 'unresolved' instructins.

def flatten_list(l):
    return [x for sublist in l for x in sublist]

completed = {}
for key in instructions.keys():
    flat_list = flatten_list(instructions[key])
    if flat_list in [['a'], ['b']]:
        completed.update({key:flat_list[0]})
completed

{'18': 'a', '39': 'b'}

In [14]:
# This just to swap in the values of things we have done
def replace_value(l, val):
    l = copy.deepcopy(l)
    for i, sublist in enumerate(l):
        for j, x in enumerate(sublist):
            if l[i][j] == val:
                l[i][j] = completed[val]
    return l

replace_value(instructions['50'], '18')

[['5', 'a'], ['61', '39']]

In [15]:
# Are we done looking at this one:
def check_done(l):
    for x in flatten_list(l):
        if re.match(r"[0-9]", x):
            return False
    return True
check_done(instructions['50'])

False

In [16]:
check_done(instructions['18'])

True

In [17]:
# Smoosh down the contents in to a regex:
def regex_or(l):
    l = [''.join(sublist) for sublist in l]
    if len(l) > 1:
        return '(' + '|'.join(l) + ')'
    else:
        return l[0]
regex_or(instructions['50'])    

'(518|6139)'

In [18]:
# FINAL VERSION OF LOOP HERE:
checked = []
# And now we loop till she done:
while not check_done(instructions['0']):
    
    # pick an element from the completed dict that we haven't checked yet:
    next_key = ''
    i = 0
    while next_key == '':
        if list(completed.keys())[i] in checked:
            i += 1
        else:
            next_key = list(completed.keys())[i]
    
    # loop through the incomplete keys
    for key in instructions.keys():
        if key in completed.keys():
            next
        # If we find a match:
        if next_key in flatten_list(instructions[key]):
            # Replace and update
            new_val = replace_value(instructions[key], next_key)
            # If it's done, add it to the list
            if check_done(new_val):
                new_val = regex_or(new_val)
                completed.update({key:new_val})
            # In any case update the instructions list
            instructions.update({key:new_val})
    # mark this off as having been done
    checked += [next_key]

In [19]:
pattern = re.compile('^' + instructions['0'] + '$')
sol1 = 0
for message in messages:
    if pattern.match(message):
        sol1 +=1
sol1

233

### Part 2
For the second part of this question

In [ ]:
messages

In [ ]:
'hi'.replace('l', '')

In [ ]:
?str.replace

In [ ]:
?re.match

In [ ]:
for i, message in enumerate(messages):
    print(i, message, bool(re.compile('^a((aa|bb)(ab|ba)|(ab|ba)(aa|bb))b$').match(messages[i])))
    

In [ ]:
for i, message in enumerate(messages):
    print(i, message, bool(re.compile('^' + instructions['0'] + '$').match(messages[i])))
    

In [ ]:
instructions['0']

In [ ]:
?re.compile